## Libraries

In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

## Dateset and Parameters

In [9]:
base_dir = "./dataset"

train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')

categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

img_size = 128
batch_size = 32
epochs = 50

## Data Preparation

In [10]:
def prepare_data(directory, categories, img_size):
    data = []
    labels = []
    for category in categories:
        path = os.path.join(directory, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = tf.keras.preprocessing.image.load_img(os.path.join(path, img), target_size=(img_size, img_size), color_mode="grayscale")
                img_array = tf.keras.preprocessing.image.img_to_array(img_array)
                img_array = img_array / 255.0
                data.append(img_array)
                labels.append(class_num)
            except Exception as e:
                pass
    return np.array(data), np.array(labels)

X_train, y_train = prepare_data(train_dir, categories, img_size)
X_test, y_test = prepare_data(test_dir, categories, img_size)

y_train = to_categorical(y_train, num_classes=len(categories))
y_test = to_categorical(y_test, num_classes=len(categories))

### Data Augmentation

In [4]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)

### Capsule Network Model

In [14]:
class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return tf.sqrt(tf.reduce_sum(tf.square(inputs), -1) + 1e-9)

class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsules, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.routings = routings

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=[input_shape[-1], self.num_capsules * self.dim_capsules], initializer='glorot_uniform', trainable=True)

    def call(self, inputs, training=None):
        inputs_expand = tf.expand_dims(inputs, axis=-1)
        inputs_tiled = tf.tile(inputs_expand, [1, 1, self.num_capsules * self.dim_capsules])
        inputs_hat = tf.keras.backend.batch_dot(inputs_tiled, self.kernel)
        inputs_hat_reshape = tf.reshape(inputs_hat, [-1, self.num_capsules, self.dim_capsules])
        
        b = tf.zeros(shape=[tf.shape(inputs_hat_reshape)[0], self.num_capsules, 1])
        
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=1)
            outputs = tf.reduce_sum(c * inputs_hat_reshape, axis=1)
            outputs = tf.nn.l2_normalize(outputs, axis=-1)
            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat_reshape * tf.expand_dims(outputs, 2), axis=-1)
        
        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsules, self.dim_capsules)

# Define the Capsule Network Model with debugging for shape
input_image = layers.Input(shape=(img_size, img_size, 1))
conv1 = layers.Conv2D(256, (9, 9), activation='relu')(input_image)
conv2 = layers.Conv2D(256, (9, 9), strides=(2, 2), activation='relu')(conv1)
primary_caps = layers.Conv2D(32 * 8, (9, 9), activation='relu')(conv2)

# Print shape of the primary_caps to debug
print("Shape of primary_caps:", primary_caps.shape)

# Adjust the reshape layer based on the printed shape
primary_caps_reshape = layers.Reshape((48 * 48 * 32, 8))(primary_caps)
primary_caps = layers.Lambda(lambda z: tf.sqrt(tf.reduce_sum(tf.square(z), -1)))(primary_caps_reshape)

capsule = CapsuleLayer(num_capsules=len(categories), dim_capsules=16)(primary_caps)
output = Length()(capsule)

model = models.Model(inputs=input_image, outputs=output)
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Shape of primary_caps: (None, 48, 48, 256)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 120, 120, 256)  │        20,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 56, 56, 256)    │     5,308,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 48, 48, 256)    │     5,308,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_5 (Reshape)             │ (None, 73728, 8)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_5 (Lambda)               │ (None, 73728)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capsule_layer_5 (CapsuleLayer)  │ (None, 4, 16)          │     4,718,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ length_1 (Length)               │ (None, 4)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,356,928 (58.58 MB)

 Trainable params: 15,356,928 (58.58 MB)

 Non-trainable params: 0 (0.00 B)

## Training the Model

In [16]:
history = model.fit(train_generator, steps_per_epoch=len(X_train) // batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Save the model
model.save("capsnet_brain_tumor_model.h5")

Epoch 1/50


ValueError: Exception encountered when calling CapsuleLayer.call().

[1mDimensions must be equal, but are 4 and 16 for '{{node functional_3_1/capsule_layer_5_1/mul_1}} = Mul[T=DT_FLOAT](functional_3_1/capsule_layer_5_1/Reshape, functional_3_1/capsule_layer_5_1/ExpandDims_2)' with input shapes: [84934656,4,16], [84934656,16,1].[0m

Arguments received by CapsuleLayer.call():
  • inputs=tf.Tensor(shape=(None, 73728), dtype=float32)
  • training=True